In [1]:
import strawberryfields as sf
from strawberryfields import ops
import numpy as np


import numpy as np


def generate_haar_unitary(dim):
    """Generate a (dim, dim) Haar-random matrix using the QR decomposition.
    Source: https://pennylane.ai/qml/demos/tutorial_haar_measure.html

    Parameters:
        dim (int): the dimension of the matrix.
    """
    # Step 1
    A, B = np.random.normal(size=(dim, dim)), np.random.normal(size=(dim, dim))
    Z = A + 1j * B

    # Step 2
    Q, R = np.linalg.qr(Z)

    # Step 3
    Lambda = np.diag([R[i, i] / np.abs(R[i, i]) for i in range(dim)])

    # Step 4
    return np.dot(Q, Lambda)


self_nb_modes = 6

self_unitary_top = generate_haar_unitary(self_nb_modes // 2)
self_unitary_bottom = generate_haar_unitary(self_nb_modes // 2)

eng = sf.Engine("fock", backend_options={"cutoff_dim": 3})
prog = sf.Program(self_nb_modes)

mid_l = int(np.floor(self_nb_modes / 2) - 1)
mid_h = int(np.floor(self_nb_modes / 2))

with prog.context as q:
    # Two-photons entanglement source
    ops.Fock(1) | q[mid_l]
    ops.Fock(1) | q[mid_h]

    ops.BSgate(phi=np.pi / 2) | (q[mid_l - 1], q[mid_l])
    ops.BSgate(phi=np.pi / 2) | (q[mid_h], q[mid_h + 1])

    ops.BSgate(theta=np.pi / 2, phi=np.pi / 2) | (q[mid_l], q[mid_h])  # SWAP

    # Interferometer
    ops.Interferometer(self_unitary_top) | q[: mid_l + 1]
    ops.Interferometer(self_unitary_bottom) | q[mid_h:]

    # Detection
    ops.MeasureFock() | q

sample = eng.run(prog).samples[0]

print(sample)

[1 0 0 0 1 0]


In [8]:
print(qml.draw(circuit)())

0: ────────────────────────────────────╭U(M0)─┤  <n>
1: ──────╭BS(0.79,1.57)────────────────├U(M0)─┤  <n>
2: ──|1⟩─╰BS(0.79,1.57)─╭BS(1.57,1.57)─╰U(M0)─┤  <n>
3: ──|1⟩─╭BS(0.79,1.57)─╰BS(1.57,1.57)─╭U(M1)─┤  <n>
4: ──────╰BS(0.79,1.57)────────────────├U(M1)─┤  <n>
5: ────────────────────────────────────╰U(M1)─┤  <n>


In [ ]:
import pennylane as qml
n_modes = self_nb_modes

dev = qml.device("strawberryfields.fock", wires=n_modes, cutoff_dim=6)

@qml.qnode(dev)
def circuit():
    mid_l = n_modes // 2 - 1
    mid_h = n_modes // 2

    # Two-photon entanglement source
    qml.FockState(1, wires=mid_l)
    qml.FockState(1, wires=mid_h)

    # Beam splitters for entanglement
    qml.Beamsplitter(phi=np.pi / 2, theta=np.pi /4, wires=[mid_l - 1, mid_l])
    qml.Beamsplitter(phi=np.pi / 2, theta=np.pi /4, wires=[mid_h, mid_h + 1])

    # SWAP operation
    #qml.SWAP(wires=[mid_l, mid_h])

    qml.Beamsplitter(phi=np.pi / 2, theta=np.pi / 2, wires=[mid_l, mid_h])
    # Interferometer (apply random Haar unitaries)
    
    qml.InterferometerUnitary(self_unitary_top, wires=list(range(mid_l + 1)))
    qml.InterferometerUnitary(self_unitary_bottom, wires=list(range(mid_h, n_modes)))

    # Measurement
    return [qml.expval(qml.NumberOperator(wires=i)) for i in range(n_modes)]

# Run the circuit and get expectation values
expectation_values = circuit()

# Post-process to simulate photon number counts
threshold = 0.5  # Threshold to consider whether a mode has a photon or not
photon_counts = [1 if ev > threshold else 0 for ev in expectation_values]

print("Photon Counts:", photon_counts)